In [ ]:
for order in orders:
    for size in mesh_size:
        eps = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)        
        ba_y.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        fes = FESpace([V, Qx, Qy])

        (us, px, py), (vs, qx, qy) = fes.TnT()
        
        #coeff=exact
        p = coeff_x * px + coeff_y * py
        q = coeff_x * qx + coeff_y * qy

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        #for i in range(5):
        #    res.data = f.vec - m.mat * gfu.vec
        #    gfu.vec.data +=  invmstar * res
        #    print(Norm(res))
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
        #Draw(u2, mesh, 'test')
        #Draw(u2-exact,mesh, 'error')
        #Draw(gfu.components[0], mesh, 'testa')
        #Draw(gfu.components[1], mesh, 'testb')
        #Draw(gfu.components[1] * coeff, mesh, 'testc')
        #input("")
        #gfu.components[0].vec[:]=0
        #for i in range(Q.ndof):
        #    gfu.components[1].vec[i]=1
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", sqrt (Integrate ((u2 - exact)*(u2- exact), mesh)))
    print('...........................................................')

In [ ]:
for val0 in range(1, 10, 1):
    for val1 in range(1, 10, 1):
        for order in orders:
            for size in mesh_size:
                eps = size
                #eps = 1e-8
                mesh = Mesh(unit_square.GenerateMesh(maxh=size))
                V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
                Q = L2(mesh, order=0)

                ba_x = BitArray(Q.ndof)        
                ba_x.Clear()
                for el in Q.Elements():
                    mark = False
                    for v in el.vertices:
                        if (mesh[v].point[0] > 1-eps):
                            mark = True
                    for dof in el.dofs:
                        ba_x[dof] = mark

                ba_y = BitArray(Q.ndof)        
                ba_y.Clear()
                for el in Q.Elements():
                    mark = False
                    for v in el.vertices:
                        if (mesh[v].point[1] > 1-eps):
                            mark = True
                    for dof in el.dofs:
                        ba_y[dof] = mark

                Qx = Compress(Q, active_dofs=ba_x)
                Qy = Compress(Q, active_dofs=ba_y)

                fes = FESpace([V, Qx, Qy])

                (us, px, py), (vs, qx, qy) = fes.TnT()

                p = coeff_x * px + coeff_y * py
                q = coeff_x * qx + coeff_y * qy

                u = us + p
                v = vs + q

                m = BilinearForm(fes, symmetric=False)
                m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,val0*order))
                m.Assemble()

                f = LinearForm(fes)
                f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,val1*order))
                f.Assemble()

                invmstar = m.mat.Inverse(inverse="sparsecholesky")

                gfu = GridFunction(fes)
                gfu.vec.data = invmstar * f.vec

                u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
                print('val0: ', val0, 'val1: ', val1, 'Order:', order, ' Mesh Size:', size , "L2 Error:", sqrt (Integrate ((u2 - exact)*(u2- exact), mesh)))
        #print('val0: ', val0, 'val1: ', val1, 'Order:', order, ' Mesh Size:', size , "L2 Error:", sqrt (Integrate ((u2 - exact)*(u2- exact), mesh)))
    print('.........................................................................')